In [44]:
from dotenv import load_dotenv
load_dotenv()
import os, json, pickle, time
import regex as re
from bot import set_driver, open_page, get_href_page, check_page
from parse import get_article_info
import pandas as pd

output_name = "test"

ut_eid = "gal767"

eid_password = os.getenv('eid_password')

driver_path = "C://Users//galon//cd_secure//chromedriver.exe"

ARTICLE_DIR = os.path.join('..//data', 'article_hrefs', 'test')

os.makedirs(ARTICLE_DIR, exist_ok=True)

In [45]:
def gen_searches():
    
    try:
        with open('step5_searches.pickle', 'rb') as file:
            articles = pickle.load(file)
            if len(articles) == 0:
                print('Create a new pickle to manage searches; current list has been exhausted.')
    except FileNotFoundError:
        articles = {}
        for file_name in os.listdir(ARTICLE_DIR):
            l = []
            with open(os.path.join(os.path.join(ARTICLE_DIR, file_name))) as file:
                href_list =  [t for t in re.sub('\n','',file.read()).split(r'"') if len(re.findall('\w+', t)) > 0]
                for href in href_list:
                    l.append(f"https://global-factiva-com.ezproxy.lib.utexas.edu/{href.split('&')[0].split('../')[1]}")
            articles[file_name.split('.')[0]] = l
    return articles

In [46]:
def search_article_list(driver, wait, article_list, name):
    
    temp_list = []
    name_list = name.split('_')
    for href in article_list:
        temp_dict = {'company_code': name_list[0], 'news_code': name_list[1], 'date': name_list[2]}
        get_href_page(driver, wait, href)
        check_page(driver, wait, href)
        temp_list.append(get_article_info(driver.page_source, temp_dict))
        time.sleep(4)
    return temp_list

In [49]:
def get_text(eid_username, eid_password, path):
    
    results = []
    searches = gen_searches()
    searches_pickle = searches.copy()
    driver, wait = set_driver(path)
    open_page(driver, wait, eid_username, eid_password)
    
    for name, article_list in searches.items():

        temp_list = search_article_list(driver, wait, article_list, name)
        if temp_list != None:
            results += temp_list
            print(results)
        try:
            searches_pickle.pop(name)
            with open('step5_searches.pickle', 'wb') as file:
                pickle.dump(searches_pickle, file)
            with open('dict_list.pickle', 'wb') as file:
                pickle.dump(results, file)
        except:
            return 'Error when saving'
    pd.DataFrame.from_dict(results).to_csv('article_test.csv')

In [50]:
get_text(ut_eid, eid_password, driver_path)

https://global-factiva-com.ezproxy.lib.utexas.edu/du/article.aspx/?accessionno=j000000020021018dyai00009
[{'company_code': 'leghot', 'news_code': 'sfwsj', 'date': '2002', 'HD': 'Fairmont Hotels to Run Monarch', 'WC': '83 words', 'PD': '18 October 2002', 'SN': 'The Wall Street Journal', 'SC': 'J', 'PG': 'B5', 'LA': 'English', 'CY': '(Copyright (c) 2002, Dow Jones & Company, Inc.)', 'LP': "TORONTO -- Fairmont Hotels & Resorts Inc.'s 35%-owned Legacy Hotels\nReal Estate Investment Trust agreed to acquire the Monarch Hotel in\nWashington, D.C., for $145 million. Fairmont said it will operate the\nhotel. Fairmont, which operates New York's Plaza Hotel and a string of\nCanadian luxury properties, has been seeking to add to its U.S. hotel\nportfolio. In August, it secured a contract to manage Sonoma Mission Inn\n& Spa in Sonoma County, Calif.", 'TD': '', 'CO': 'farmnt : Fairmont Hotels & Resorts Inc | leghot : Legacy Hotels Real Estate Investment Trust', 'IN': 'i66 : Hotels/Restaurants/Casino